### Information retrieval project

Goal is to create an IR model that can perform both boolean (AND, OR and NOT), wildcard and phrase queries.

Perform normalization and stemming.

Spelling correction.

Evaluate the system on test queries.

In [37]:
import numpy as np
import pandas as pd


import re
import json
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

from nltk import word_tokenize

import string


In [38]:
def read_documents():
    f = open("archive/CISI.ALL")
    merged = " "
    i = 0
    for a_line in f.readlines ():
            if a_line.startswith ("."):
                i += 1
                merged += "\n" + a_line.strip ()
            else:
                i += 1
                merged += " " + a_line.strip ()
        # updates the merged variable using a for-loop
    documents = {}
    content = ""
    doc_id = ""
    # each entry in the dictioanry contains key = doc_id and value = content

    for line in merged.split ("\n"):
        #print(a_line)
        if line.startswith (".I"):
            doc_id = line.split (" ") [1].strip()
        elif line.startswith (".X"):
            documents[doc_id] = content
            content = ""
            doc_id = ""
        else:
            content += line.strip ()[3:] + " "
            #Extract after . a letter and a space
    f.close ()
    return documents


In [39]:
def read_queries():
    f = open("archive/CISI.QRY")

    merged = ""

    for line in f.readlines ():
        if line.startswith ("."):
            merged += "\n" + line.strip ()
        else:
            merged += " " + line.strip ()
    
    queries = {}

    content = ""
    qry_id = ""

    for line in merged.split ("\n"):
        if line.startswith(".I"):
            if not content == "":
                queries [qry_id] = content
                content = ""
                qry_id = ""
            # add an enrty to the dictionary when you encounter an .I identifier
            qry_id = line.split(" ")[1].strip ()
        # otherwise, keep adding content to the content variable
        elif line.startswith (".W") or line.startswith (".T"):
            content += line.strip ()[3:] + " "
    queries [qry_id] = content
    f.close ()
    return queries


In [40]:
def read_relevance():
    f = open("archive/CISI.REL")
    mappings = {}
    
    for line in f.readlines ():
        voc = line.strip ().split ()
        key = voc[0].strip ()
        current_value = voc[1].strip()
        value = []
        # update the entry in the mappings dictionary with the current value
        if key in mappings.keys ():
            value = mappings.get (key)
        value.append (current_value)
        mappings [key] = value
    f.close ()
    return mappings

In [41]:
documents = read_documents()
print(len(documents))
print(documents["1"])


1460
 18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


In [42]:
queries = read_queries()
print(len(queries))
print(queries["1"])

112
What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles? 


In [43]:
relevance = read_relevance()
print(len(relevance))
print(relevance["1"])


76
['28', '35', '38', '42', '43', '52', '65', '76', '86', '150', '189', '192', '193', '195', '215', '269', '291', '320', '429', '465', '466', '482', '483', '510', '524', '541', '576', '582', '589', '603', '650', '680', '711', '722', '726', '783', '813', '820', '868', '869', '894', '1162', '1164', '1195', '1196', '1281']


In [44]:
def get_terms(text):
    terms = {}
    ps = PorterStemmer()
    word_list = [ps.stem(word) for word in word_tokenize(text.lower()) if not word in string.punctuation]
    #print(word_list)
    for word in word_list:
        terms[word] = terms.get(word, 0) + 1
    return terms

doc_terms = {}
qry_terms = {}
for doc_id in documents.keys ():
    text = documents.get (doc_id)
    #print(word_tokenize(text.lower()))
    doc_terms[doc_id] = get_terms(documents.get(doc_id))

for qry_id in queries.keys ():
    # populate the term frequency dictionaries for all documents and all queries
    qry_terms [qry_id] = get_terms (queries.get (qry_id))


In [45]:
print (len (doc_terms))
print (doc_terms.get ("1"))
print (len (doc_terms.get("1")))
print (len (qry_terms))
print (qry_terms.get("1"))
print (len (qry_terms.get("1")))

1460
{'18': 1, 'edit': 4, 'of': 7, 'the': 10, 'dewey': 3, 'decim': 2, 'classif': 2, 'comaromi': 1, 'j.p.': 1, 'present': 1, 'studi': 1, 'is': 2, 'a': 2, 'histori': 2, 'first': 2, 'ddc': 2, 'wa': 1, 'publish': 1, 'in': 4, '1876': 1, 'eighteenth': 1, '1971': 1, 'and': 3, 'futur': 1, 'will': 1, 'continu': 1, 'to': 2, 'appear': 1, 'as': 1, 'need': 1, 'spite': 1, "'s": 1, 'long': 1, 'healthi': 1, 'life': 1, 'howev': 1, 'it': 1, 'full': 1, 'stori': 1, 'ha': 2, 'never': 1, 'been': 2, 'told': 1, 'there': 1, 'have': 1, 'biographi': 1, 'that': 2, 'briefli': 1, 'describ': 1, 'hi': 1, 'system': 1, 'but': 1, 'thi': 2, 'attempt': 1, 'provid': 1, 'detail': 1, 'work': 1, 'more': 1, 'than': 1, 'ani': 1, 'other': 1, 'spur': 1, 'growth': 1, 'librarianship': 1, 'countri': 1, 'abroad': 1}
66
112
{'what': 3, 'problem': 1, 'and': 1, 'concern': 1, 'are': 2, 'there': 1, 'in': 2, 'make': 1, 'up': 1, 'descript': 1, 'titl': 3, 'difficulti': 1, 'involv': 1, 'automat': 1, 'retriev': 1, 'articl': 2, 'from': 1, 'appr

In [46]:
def jaccard_similarity(query, document):
    query_set = set(query)
    doc_set = set(document)
    return len(query_set & doc_set) / len(query_set | doc_set)


print(jaccard_similarity(qry_terms.get("1"), doc_terms.get("35")))  # Output vicino a 1 indica alta rilevanza


0.10526315789473684


In [47]:
for i in range(1, 30):
    print(jaccard_similarity(qry_terms.get("1"), doc_terms.get(str(i))))  # Output vicino a 1 indica alta rilevanza

0.08235294117647059
0.1
0.08536585365853659
0.05747126436781609
0.06299212598425197
0.059602649006622516
0.08641975308641975
0.07586206896551724
0.05785123966942149
0.07407407407407407
0.10975609756097561
0.06153846153846154
0.07228915662650602
0.06722689075630252
0.08108108108108109
0.06832298136645963
0.05439330543933055
0.0761904761904762
0.051470588235294115
0.06363636363636363
0.057971014492753624
0.07317073170731707
0.08333333333333333
0.1388888888888889
0.08163265306122448
0.10975609756097561
0.0425531914893617
0.08609271523178808
0.125


In [48]:
## INVERTED INDEX
def normalize(text):
    no_punctuation = re.sub(r'[^\w^\s*-]','',text) # remove punctuation
    downcase = no_punctuation.lower() # lowercase
    return downcase

def tokenize(content):
    text = normalize(content)
    return list(text.split()) # return a list of tokens

def Lstemm(content):
    ps = LancasterStemmer() # stemmer
    text = tokenize(content) # tokenize
    return list(set([ps.stem(word) for word in text]))
def Pstemm(content):
    ps = PorterStemmer() # stemmer
    text = tokenize(content) # tokenize
    return list(set([ps.stem(word) for word in text])) 

In [49]:
# create inverted index
def create_inverted_index_no_norm(documents):
    inverted_index = {}
    for doc_id, content in documents.items():
        for token in content.split():
            if token in inverted_index.keys():
                if doc_id not in inverted_index[token]:
                    inverted_index[token].append(doc_id)
            else:
                inverted_index[token] = [doc_id]
        #if (int(doc_id) % 100 == 0):
        #    print("ID: " + str(doc_id))
    return inverted_index

In [50]:
# create inverted index
def create_inverted_index_P(documents):
    inverted_index = {}
    for doc_id, content in documents.items():
        #print(content)
        for token in Pstemm(content):
            if token in inverted_index.keys():
                if doc_id not in inverted_index[token]:
                    inverted_index[token].append(doc_id)
            else:
                inverted_index[token] = [doc_id]
        #if (int(doc_id) % 100 == 0):
        #    print("ID: " + str(doc_id))
    return inverted_index

In [51]:
# create inverted index
def create_inverted_index_L(documents):
    inverted_index = {}
    for doc_id, content in documents.items():
        #print(content)
        for token in Lstemm(content):
            if token in inverted_index.keys():
                if doc_id not in inverted_index[token]:
                    inverted_index[token].append(doc_id)
            else:
                inverted_index[token] = [doc_id]
        #if (int(doc_id) % 100 == 0):
        #    print("ID: " + str(doc_id))
    return inverted_index

In [55]:
inv_index_no_norm = create_inverted_index_no_norm(documents)

inv_index_L = create_inverted_index_L(documents)

inv_index_P = create_inverted_index_P(documents)

print(f"Len no norm: {len(inv_index_no_norm)}")
print(f"Len L: {len(inv_index_L)}")
print(f"Len P: {len(inv_index_P)}")

Len no norm: 21967
Len L: 7556
Len P: 8554


In [ ]:
def order_inverted_index(inverted_index):
    ordered_inverted_index = {}
    for key in sorted(inverted_index.keys()):
        ordered_inverted_index[key] = inverted_index[key]
    return ordered_inverted_index

ordered = order_inverted_index(inv_index_no_norm)

In [57]:
inv_index_P.get("class")

['5',
 '16',
 '42',
 '176',
 '233',
 '275',
 '282',
 '290',
 '328',
 '341',
 '345',
 '363',
 '379',
 '404',
 '405',
 '417',
 '428',
 '455',
 '476',
 '478',
 '479',
 '486',
 '559',
 '577',
 '610',
 '669',
 '694',
 '701',
 '722',
 '769',
 '791',
 '797',
 '798',
 '838',
 '857',
 '862',
 '945',
 '954',
 '958',
 '1029',
 '1075',
 '1180',
 '1204',
 '1217',
 '1237',
 '1380',
 '1395',
 '1398',
 '1415',
 '1423']

In [63]:
import re

def tokenize_query(query):
    """
    Splits the query into tokens (terms, operators, parentheses).
    Operators are normalized to uppercase and terms to lowercase.
    """
    # This regex captures words, parentheses, and the Boolean operators.
    tokens = re.findall(r'\(|\)|\bAND\b|\bOR\b|\bNOT\b|\w+', query, flags=re.IGNORECASE)
    normalized_tokens = []
    for token in tokens:
        if token.upper() in {"AND", "OR", "NOT"}:
            normalized_tokens.append(token.upper())
        elif token in {"(", ")"}:
            normalized_tokens.append(token)
        else:
            normalized_tokens.append(token.lower())
    return normalized_tokens

def shunting_yard(tokens):
    """
    Converts infix expression tokens to postfix (RPN) using the Shunting-yard algorithm.
    Operator precedence: NOT > AND > OR.
    """
    output = []
    op_stack = []
    precedence = {"NOT": 3, "AND": 2, "OR": 1}
    
    for token in tokens:
        # Check if token is an operator first.
        if token in {"AND", "OR", "NOT"}:
            # Pop operators with higher or equal precedence from the op_stack.
            while (op_stack and op_stack[-1] != "(" and 
                   op_stack[-1] in precedence and 
                   precedence[op_stack[-1]] >= precedence[token]):
                output.append(op_stack.pop())
            op_stack.append(token)
        elif token == "(":
            op_stack.append(token)
        elif token == ")":
            # Pop until an '(' is encountered.
            while op_stack and op_stack[-1] != "(":
                output.append(op_stack.pop())
            if op_stack and op_stack[-1] == "(":
                op_stack.pop()  # Remove the '('
            else:
                raise ValueError("Mismatched parentheses in query.")
        else:
            # The token is a term.
            output.append(token)
    
    # Append any remaining operators.
    while op_stack:
        op = op_stack.pop()
        if op in {"(", ")"}:
            raise ValueError("Mismatched parentheses in query.")
        output.append(op)
    
    return output

def evaluate_postfix(postfix_tokens, inverted_index, universal_set):
    """
    Evaluates the Boolean query given in postfix notation.
    Returns a set of document IDs matching the query.
    """
    stack = []
    for token in postfix_tokens:
        if token in {"AND", "OR", "NOT"}:
            if token == "NOT":
                if not stack:
                    raise ValueError("Insufficient operands for NOT operator.")
                operand = stack.pop()
                result = universal_set - operand
                stack.append(result)
            else:
                if len(stack) < 2:
                    raise ValueError(f"Insufficient operands for {token} operator.")
                right = stack.pop()
                left = stack.pop()
                if token == "AND":
                    result = left & right  # Intersection
                elif token == "OR":
                    result = left | right  # Union
                stack.append(result)
        else:
            # The token is a term.
            posting = inverted_index.get(token, set())
            stack.append(posting)
    
    if len(stack) != 1:
        raise ValueError(f"Error in evaluation: stack should have exactly one element, but got {stack}")
    
    return stack[0]

def evaluate_boolean_query(query, inverted_index, universal_set):
    """
    Processes a Boolean query:
      1. Tokenizes the query.
      2. Converts it to postfix notation.
      3. Evaluates the postfix expression.
    Returns a set of document IDs that satisfy the query.
    """
    tokens = tokenize_query(query)
    # Debug: Uncomment to see tokens
    # print("Tokens:", tokens)
    
    postfix = shunting_yard(tokens)
    # Debug: Uncomment to see postfix notation
    # print("Postfix:", postfix)
    
    result = evaluate_postfix(postfix, inverted_index, universal_set)
    return result

# ------------------------------
# Example Usage
# ------------------------------

if __name__ == "__main__":
    # Sample inverted index: term -> set of document IDs.
    inverted_index = {
        "apple": {1, 2, 4},
        "banana": {2, 3},
        "cherry": {1, 3, 5},
        "date": {3, 6}
    }

    # Universal set: all document IDs.
    universal_set = {1, 2, 3, 4, 5, 6}

    # List of example Boolean queries.
    queries = [
        "apple AND (banana OR cherry)",   # Expected: Intersection of {1,2,4} and union of {2,3} and {1,3,5} -> {1,2}
        "apple OR banana",                  # Expected union: {1,2,3,4}
        "apple AND NOT banana",             # Expected: {1,2,4} - {2,3} -> {1,4}
        "NOT cherry",                       # Expected: universal_set - {1,3,5} -> {2,4,6}
        "(apple OR banana) AND (cherry OR date)"
    ]

    for query in queries:
        try:
            result = evaluate_boolean_query(query, inverted_index, universal_set)
            print(f"Query: {query}\nMatching Documents: {result}\n")
        except ValueError as ve:
            print(f"Query: {query}\nError: {ve}\n")


Query: apple AND (banana OR cherry)
Matching Documents: {1, 2}

Query: apple OR banana
Matching Documents: {1, 2, 3, 4}

Query: apple AND NOT banana
Matching Documents: {1, 4}

Query: NOT cherry
Matching Documents: {2, 4, 6}

Query: (apple OR banana) AND (cherry OR date)
Matching Documents: {1, 3}



In [ ]:
import re
from nltk.stem import PorterStemmer

# Initialize the Porter Stemmer
stemmer = PorterStemmer()

def tokenize_query(query):
    """
    Splits the query into tokens (terms, operators, and parentheses).
    Operators are normalized to uppercase.
    Terms are lowercased and stemmed using the Porter Stemmer.
    """
    # This regex captures words, parentheses, and Boolean operators.
    tokens = re.findall(r'\(|\)|\bAND\b|\bOR\b|\bNOT\b|\w+', query, flags=re.IGNORECASE)
    normalized_tokens = []
    for token in tokens:
        # Check if token is an operator
        if token.upper() in {"AND", "OR", "NOT"}:
            normalized_tokens.append(token.upper())
        elif token in {"(", ")"}:
            normalized_tokens.append(token)
        else:
            # For terms, lowercase and apply stemming
            normalized_tokens.append(stemmer.stem(token.lower()))
    return normalized_tokens

def shunting_yard(tokens):
    """
    Converts infix expression tokens to postfix (RPN) using the Shunting-yard algorithm.
    Operator precedence: NOT > AND > OR.
    """
    output = []
    op_stack = []
    precedence = {"NOT": 3, "AND": 2, "OR": 1}
    
    for token in tokens:
        if token in {"AND", "OR", "NOT"}:
            # Pop operators with higher or equal precedence
            while (op_stack and op_stack[-1] != "(" and 
                   op_stack[-1] in precedence and 
                   precedence[op_stack[-1]] >= precedence[token]):
                output.append(op_stack.pop())
            op_stack.append(token)
        elif token == "(":
            op_stack.append(token)
        elif token == ")":
            # Pop until an '(' is encountered
            while op_stack and op_stack[-1] != "(":
                output.append(op_stack.pop())
            if op_stack and op_stack[-1] == "(":
                op_stack.pop()  # Remove the '('
            else:
                raise ValueError("Mismatched parentheses in query.")
        else:
            output.append(token)
    
    while op_stack:
        op = op_stack.pop()
        if op in {"(", ")"}:
            raise ValueError("Mismatched parentheses in query.")
        output.append(op)
    
    return output

def evaluate_postfix(postfix_tokens, inverted_index, universal_set):
    """
    Evaluates the Boolean query given in postfix notation.
    Returns a set of document IDs matching the query.
    """
    stack = []
    for token in postfix_tokens:
        if token in {"AND", "OR", "NOT"}:
            if token == "NOT":
                if not stack:
                    raise ValueError("Insufficient operands for NOT operator.")
                operand = stack.pop()
                result = universal_set - operand
                stack.append(result)
            else:
                if len(stack) < 2:
                    raise ValueError(f"Insufficient operands for {token} operator.")
                right = stack.pop()
                left = stack.pop()
                if token == "AND":
                    result = left & right  # Intersection
                elif token == "OR":
                    result = left | right  # Union
                stack.append(result)
        else:
            posting = inverted_index.get(token, set())
            stack.append(posting)
    
    if len(stack) != 1:
        raise ValueError(f"Error in evaluation: stack should have exactly one element, but got {stack}")
    
    return stack[0]

def evaluate_boolean_query(query, inverted_index, universal_set):
    """
    Processes a Boolean query:
      1. Tokenizes the query (with stemming).
      2. Converts it to postfix notation.
      3. Evaluates the postfix expression.
    Returns a set of document IDs that satisfy the query.
    """
    tokens = tokenize_query(query)
    postfix = shunting_yard(tokens)
    result = evaluate_postfix(postfix, inverted_index, universal_set)
    return result 

if __name__ == "__main__":

    universal_set = set(range(1, 1461))


    queries = [
        "NOT class AND NOT game"
    ]
    converted_index = {term: set(map(int, doc_ids)) for term, doc_ids in inv_index_P.items()}

    for query in queries:
        try:
            result = evaluate_boolean_query(query, converted_index, universal_set)
            print(f"Query: {query}\nMatching Documents: {result}\n")
        except ValueError as ve:
            print(f"Query: {query}\nError: {ve}\n")


Query: NOT class AND NOT game
Matching Documents: {1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216